# **Import Libraries**

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns

## **Load Data**

In [36]:
df = pd.read_csv('../input/sf-salaries/Salaries.csv')

## **Overview**

In [37]:
df.head()

In [38]:
# general information

# total 13 columns
# total 148 654 rows
# incorrect data types
# missing exist
# unnecessary columns

df.info()

In [39]:
# statistical information

df.describe()

## **Data Cleaning**

###  Modify data types in columns

In [40]:
# convert numeric columns to float instead of object.
df['BasePay'] = pd.to_numeric(df['BasePay'],errors='coerce')
df['OvertimePay'] = pd.to_numeric(df['OvertimePay'],errors='coerce')
df['OtherPay'] = pd.to_numeric(df['OtherPay'],errors='coerce')
df['Benefits'] = pd.to_numeric(df['Benefits'],errors='coerce')
df['TotalPay'] = pd.to_numeric(df['TotalPay'],errors='coerce')
df['TotalPayBenefits'] = pd.to_numeric(df['TotalPayBenefits'],errors='coerce')

# not necessary
# convert Year column from integer to datetime type
# df['Year'] = pd.to_datetime(df.Year, format='%Y')

# convert text column to string
df['EmployeeName'] = df['EmployeeName'].astype('string')
df['JobTitle'] = df['JobTitle'].astype('string')
df['Agency'] = df['Agency'].astype('string')

# confirm changes
df.info()

### Missing Values

In [41]:
# missing values

df.isnull().sum()

In [42]:
# ['Notes', 'Status'] columns is full of missing values and irreplaceable values, so we going to drop it.

df.drop(columns = ['Notes','Status'], inplace=True)

In [43]:
# The number of missing values in BasePay is relatively few so it would be appropriate to fill it in with an average.

base_pay_mean = df.BasePay.mean()
df['BasePay'] = df['BasePay'].fillna(base_pay_mean)

In [44]:
# fill NaN in Benefits with zero

df['Benefits'] = df['Benefits'].fillna(0.00)
df['OvertimePay'] = df['Benefits'].fillna(0.00)
df['OtherPay'] = df['Benefits'].fillna(0.00)

In [45]:
# confirm changes

df.isnull().sum()

### Dyplicate values

In [46]:
# check duplicate values
# no duplicate

df.duplicated().sum()

### Detect and remove outlires

In [47]:
# detecting and remove outlires

sns.boxplot(df['TotalPayBenefits']);

In [48]:
# outliers are values greater than (Q3+1.5*IQR)
Q1 = np.percentile(df['TotalPayBenefits'], 25, interpolation = 'midpoint')
Q3 = np.percentile(df['TotalPayBenefits'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1

In [49]:
# Upper bound
upper = np.where(df['TotalPayBenefits'] >= (Q3+1.5*IQR))
# lower bound
lower = np.where(df['TotalPayBenefits'] <= (Q3-1.5*IQR))


In [50]:
# Because we may need the outliers to understand a certain class of people who get very high salaries.
# it would be better to save the old data before removing the outliers (df) and store the new data in new dataframe (df_new)

df_new = df.copy()

In [51]:
# Remove outlires
df_new.drop(upper[0], inplace = True)
df_new.drop(lower[0], inplace = True)

In [52]:
# confirm changes after remove outlires
sns.boxplot(df_new['TotalPayBenefits']);

### Incorrect records

In [53]:
# remove records with BasePay = 0

df[df.BasePay == 0].BasePay.count()
# there is 1298 recordf with BasePay = 0

In [54]:
# drop it
df = df.drop(df[df.BasePay == 0].index)

### Data after cleaning

In [ ]:
df.to_csv('df_clean.csv', index=False)